In [395]:
import utility
import pandas as pd
import urllib.request
import json 
from bs4 import BeautifulSoup
import requests
import json
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
from datetime import datetime, timedelta, date
import pytz
import dateutil.parser
from collections import defaultdict
import random
import json
import csv
from dateutil.parser import parse
import os
import datetime

In [396]:
def get_unannotated_data(combined_df, annotated_df, indicator, source, project_path):
    '''get unannotated data (remove annotated data from all the articles that are collected)'''

    total = combined_df
    drop_list = annotated_df.index.values.tolist()
    total = total.drop(drop_list)
    total['title_desc'] = total['title'] + '. ' + total['description']
    total = total[['source', 'title_desc', 'publishedAt']]
    total = total.drop_duplicates(subset='title_desc', keep='first' )
    
    prediction_file_path =  "sentiment_analyzer/data/predictions_data/"
    
    if utility.check_dir_exists(project_path +prediction_file_path + source):
        try:
            prediction_file_path_with_source = prediction_file_path + source + "/"
            prediction_file_name = "predictions_dataset_" + indicator + "_" + source + '.csv'
            write_df_to_csv(total,project_path,prediction_file_path_with_source,prediction_file_name)
        except:
            print("unable to write to file \n")
            return False
        
    if utility.file_exists(project_path+prediction_file_path_with_source+prediction_file_name):
        return True
    else:
        print("Final directory is not created \n")
        return False


In [397]:

def write_df_to_csv(df,project_path,file_path,file_name):
    """Takes a dataframe and writes to a file"""
    if os.path.isdir(project_path+file_path):
        df.to_csv(project_path+file_path+file_name, encoding='utf-8', index=False)
        print(project_path+file_path+file_name, "successfully written \n")
    else:
        print("Path does not exist:", project_path+file_path + "\n")
        return None

In [398]:
def cbc_sampling_wrapper(economic_indicator,project_path, column_names_list, sample_size,source_string, remove_Nans = True):
    """This function creates a sample from a given file  of json 
    Input:
    ------
    economic_indicator: The economic indicator
    project_path: The path of the project
    json_file_path: The path tto the file which has the article jsons
    column_names_list: The list of column names to retain in the final sampled file
    sample_size: The number of articles to sample per month in any year
    source_string: The source (CBC/Bloomberg)"""
    try:
        json_file_path =  "data_extraction/data/unannotated_data/" + source_string.upper() + "/" +  economic_indicator + "_" + source_string.upper() + "_article.json"
        print("Reading json file :", project_path + json_file_path + "\n")
        
        df = utility.convert_json_to_df(project_path, json_file_path)
        
        preprocessed_df = utility.preprocess_df(df,column_names_list)
        preprocessed_df["source"] = source_string
        preprocessed_df = utility.apply_lambda(preprocessed_df,'publishedAt' ,"lambda x: x[1]")
        
        sampled_df = utility.sample_dataframe_by_month(preprocessed_df, sample_size)
        sampled_df = utility.apply_lambda(sampled_df,'publishedAt',"lambda x: dateutil.parser.isoparse(x).date()")
        
        # Write sampled dataframe to a different file in a different path
        output_file_path = "data_extraction/data/annotated_data/"  + source_string + "/"
        write_df_to_csv(sampled_df,project_path,output_file_path,economic_indicator+"_sample.csv")
        
        if utility.check_dir_exists(project_path + output_file_path +economic_indicator+"_sample.csv"):
            get_unannotated_data(preprocessed_df, sampled_df, economic_indicator, source_string, project_path)
            print("Sample and predictions ready \n")
            return True
        else:
            print("Something went wrong")
    except:
        print("Could not successfully sample the data")
        return False
    
project_path = "/Users/nagasiri/Desktop/NagaSiri/MDS-CL/Capstone/better_dwelling_capstone/project/"
c = cbc_sampling_wrapper("mortgagerates",project_path,['title', 'description','publishedAt'], 2,"cbc")

Reading json file : /Users/nagasiri/Desktop/NagaSiri/MDS-CL/Capstone/better_dwelling_capstone/project/data_extraction/data/unannotated_data/CBC/mortgagerates_CBC_article.json

/Users/nagasiri/Desktop/NagaSiri/MDS-CL/Capstone/better_dwelling_capstone/project/data_extraction/data/annotated_data/cbc/mortgagerates_sample.csv successfully written 

/Users/nagasiri/Desktop/NagaSiri/MDS-CL/Capstone/better_dwelling_capstone/project/sentiment_analyzer/data/predictions_data/cbc/predictions_dataset_mortgagerates_cbc.csv successfully written 

Sample and predictions ready 

